# Inference Tools

> This module provides a comprehensive collection of tools tailored for the analysis and inference of isomatrix anndata objects. These tools facilitate the extraction of meaningful insights from complex datasets, enabling advanced data interpretation in the context of single cell resoloution isoform studies. 


In [1]:
#| default_exp inference

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
#| export
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np

In [4]:
#| hide
import nbdev; nbdev.nbdev_export()